In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
size = 20
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    *(pds.random().alias(f"var{i}") for i in range(2)),
    pds.random_int(0, 6).alias("can_select"),
    pl.col("id").cast(pl.UInt32)
)
df.head()

In [ ]:
df_test = df.lazy().select(
    "id",
    "can_select",
    pl.when(
        pl.col("can_select") == 1
    ).then(
        pds.query_knn_ptwise(
            *(pl.col(f"var{i}") for i in range(2)), # Columns used as the coordinates in n-d space
            index = "id",  # pl.col("id"), str | pl.Expr
            k = 5000, 
            dist = "l2", # squared l2
            parallel = True
        )
    ).otherwise(
        None
    )    
    .alias("best friends")
).collect()

In [ ]:
df_test

In [ ]:
df_test.filter(
    pl.col("can_select") == 1
)

In [ ]:
pl.show_versions()